In [1]:
from scipy.io import mminfo,mmread,mmwrite
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [2]:
M = mmread('test_mass.mtx').toarray()
K = mmread('test_stiffness.mtx').toarray()

N = M.shape[0] # number of dofs
M.shape,K.shape

FileNotFoundError: [Errno 2] No such file or directory: 'test_mass.mtx'

In [ ]:
from scipy.linalg import eigh

k = 15
W,V = eigh(K,M,eigvals=(0,k))
np.sqrt(W)

In [ ]:
# plot the structure of the matrix
def matplot(M,frac=0.2) :
    from matplotlib.pyplot import subplots, cm
    from numpy import max, abs
    lim = max(abs(M))*frac
    fig,ax = subplots()
    cax = ax.matshow(M,cmap=cm.RdBu_r,vmin=-lim,vmax=lim)
    fig.colorbar(cax,extend='both')
    return fig,ax

matplot(M)
matplot(K)

In [ ]:
# Test the order of coorniates with respect to the ordering in the system matrix:
# - compare system-result (test_mode_i.vec) with result from hdf5
i = 6 # can only check this for distinct modes (non-zero frequency, i>=6)
import h5py
h5 = h5py.File('results_hdf5/JOB.cfs')
coord = h5['Mesh/Nodes/Coordinates'].value

# -> the coordinates are written in reverse order with respect to the system matrices
# -> the system matrix result vector is 1_x,1_y,1_z,2_x, ..., N_z
# -> coord order in h5 is N, N-1, 
X = coord[::-1,:]

# load real value
mode_h5 = h5['Results/Mesh/MultiStep_1/Step_%i/mechDisplacement/V_plate/Nodes/Real'%i].value
h5.close()


mode_vec = np.loadtxt('test_mode_%i.vec'%im) 
# contains real and imaginary part in columns
# -> the system matrix result vector is 1_x,1_y,1_z,2_x, ..., N_z

# check if solutions are equal
np.allclose( mode_h5[::-1,:].ravel(), mode_vec[:,0] )

In [ ]:
# resort everything
nprec = 6
# get grid vectors (the unique vectors of the x,y,z coodinate-grid)
x = np.unique(np.round(coord[:,0],decimals=nprec))
y = np.unique(np.round(coord[:,1],decimals=nprec))
z = np.unique(np.round(coord[:,2],decimals=nprec))
print('Nx=',len(x))
print('Ny=',len(y))
print('Nz=',len(z))
# find nodes in grid
Xg,Yg,Zg  = np.meshgrid(x,y,z)
coord_s = np.array([Xg.ravel(),Yg.ravel(),Zg.ravel()]).T
Is = np.zeros([coord.shape[0]],dtype=int)
for i,(xi,yi,zi) in enumerate(coord_s):
    #print(np.array([coord[:,0]-x,coord[:,1]-y,coord[:,2]-z]).shape)
    #print(np.argwhere(np.abs(coord[:,0]-x)<=np.power(10,-nprec)))
    idx = np.argwhere(np.all(np.abs(np.array([X[:,0]-xi,X[:,1]-yi,X[:,2]-zi]))<=np.power(10.,-nprec),axis=0))
    if len(idx)==0 :
        print('error:',i,xi,yi,zi)
    else :
        Is[i] = idx[0][0]
        
# for the system matrix 
Im = np.array([Is*3,Is*3+1,Is*3+2]).T.ravel()

# resosrt
K = K[Im,:][:,Im]
M = M[Im,:][:,Im]
X = X[Is,:]

In [ ]:
# save
mmwrite('../exercises/M.mtx',M,comment='NxN mass matrix with DoF order 1_x,1_y,1_z,2_x, ..., N_z')
mmwrite('../exercises/K.mtx',K,comment='NxN stiffness matrix with DoF order: 1_x,1_y,1_z,2_x, ..., N_z')
mmwrite('../exercises/X.mtx',X,comment='Nx3 coordinate matrix with columns containing the x,y,z coordinates for DoFs 1,...,N')

In [ ]:
from scipy import sparse
Ks = sparse.csr_matrix(K)
Ms = sparse.csr_matrix(M)
mmwrite('../exercises/Ms.mtx',Ms,comment='NxN mass matrix with DoF order 1_x,1_y,1_z,2_x, ..., N_z')
mmwrite('../exercises/Ks.mtx',Ks,comment='NxN stiffness matrix with DoF order 1_x,1_y,1_z,2_x, ..., N_z')

In [ ]:
matplot(M)
matplot(K)

## ToDo

* animate modes
* plot middle nodes in 3d
* implement constraint equations
* 